In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sklm
import xgboost as xgb
import matplotlib

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
train = pd.read_csv('trainreg.csv')
test = pd.read_csv('test.csv')
submissions = pd.read_csv('submissionfile.csv')
vd = pd.read_csv('VariableDefinitions.csv')

In [3]:
from sklearn import preprocessing

def encode_string(data):
    enc = preprocessing.LabelEncoder()
    enc.fit(data)
    enc_features = enc.transform(data)
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_features.reshape(-1,1))
    return encoded.transform(enc_features.reshape(-1,1)).toarray()

In [4]:
categorical_columns = ['location_type', 'cellphone_access', 'gender_of_respondent', 'relationship_with_head',
                       'marital_status', 'education_level', 'job_type']

Features_enc = encode_string(train['country'])
for col in categorical_columns:
    temp = encode_string(train[col])
    Features_enc = np.concatenate([Features_enc, temp], axis = 1)
    
print(Features_enc.shape)

test_enc = encode_string(test['country'])
for col in categorical_columns:
    temp1 = encode_string(test[col])
    test_enc = np.concatenate([test_enc, temp1], axis = 1)
    
print(test_enc.shape)

(23524, 37)
(10086, 37)


In [5]:
Features_enc = np.concatenate([Features_enc, np.array(train[['household_size', 'age_of_respondent', 'year']])], axis = 1)
print(Features_enc.shape)

test_enc = np.concatenate([test_enc, np.array(test[['household_size', 'age_of_respondent', 'year']])], axis = 1)
print(test_enc.shape)

(23524, 40)
(10086, 40)


In [6]:
scaler = preprocessing.StandardScaler().fit(Features_enc[:, 37:40])
Features_enc[:, 37:40] = scaler.transform(Features_enc[:, 37:40])
print(Features_enc[:5, 35:40])

[[ 0.          1.         -0.35800673 -0.89618796  1.20854126]
 [ 0.          0.          0.53983446  1.88827897  1.20854126]
 [ 0.          1.          0.53983446 -0.77512418  1.20854126]
 [ 0.          0.          0.53983446 -0.29086906  1.20854126]
 [ 0.          0.          1.88659625 -0.77512418  1.20854126]]


In [7]:
scaler1 = preprocessing.StandardScaler().fit(test_enc[:, 37:40])
test_enc[:, 37:40] = scaler1.transform(test_enc[:, 37:40])
print(test_enc[:5, 35:40])

[[ 0.          0.         -0.35198335 -0.51067812  1.20863498]
 [ 0.          0.          1.45583541  0.78010077  1.20863498]
 [ 1.          0.         -0.35198335  2.37820797  1.20863498]
 [ 1.          0.          1.00388072  0.04251284  1.20863498]
 [ 1.          0.         -0.35198335 -1.37119738  1.20863498]]


In [8]:
Labels = np.array(train['bank_account'])
print(Labels)

[ 46.       47.       48.      ... 715.88328 716.88328 717.88328]


In [9]:
import numpy.random as nr
import sklearn.model_selection as ms
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features_enc.shape[0])
indx = ms.train_test_split(indx, test_size = 0.2)
x_train = Features_enc[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
x_test = Features_enc[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

In [10]:
indx1 = range(x_train.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.2)
x_train1 = x_train[indx1[0],:]
y_train1 = np.ravel(y_train[indx1[0]])
x_train_val = x_train[indx1[1],:]
y_train_val = np.ravel(y_train[indx1[1]])

In [11]:
print(x_train.shape)
print(x_train1.shape)
print(x_train_val.shape)
print(y_train.shape)
print(y_train1.shape)
print(y_train_val.shape)
print(x_test.shape)
print(y_test.shape)

(18819, 40)
(15055, 40)
(3764, 40)
(18819,)
(15055,)
(3764,)
(4705, 40)
(4705,)


In [12]:
xg_cl1 = xgb.XGBRegressor(objective = 'reg:logistic', n_estimators = 4000, seed = 123, max_depth = 8,
                           learning_rate=0.01, booster = 'gbtree', base_score = 0.8, subsample = 0.8,
                           reg_lambda = 0.03)

In [13]:
def print_format(f, x, y, z):
    print('Fold %1d     %4.3f         %4.3f        %4.3f'  %  (f, x, y, z))
    
def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                  Confusion matrix')
    print('                 Score positive     Score negative')
    print('True positive    %6d' % conf[0,0] + '            %5d' % conf[0,1])
    print('True negative    %6d' % conf[1,0] + '            %5d' % conf[1,1])
    print('')
    print('Incorrect    %0.2f' % (1 - (sklm.accuracy_score(labels, scores))))
    print('')
    print('              Positive    Negative')
    print('Num case     %0.2f' % metrics[3][0] + '       %0.2f' % metrics[3][1])
    print('Precision       %0.2f' % metrics[0][0] + '       %0.2f' % metrics[0][1])
    print('Recall          %0.2f' % metrics[1][0] + '       %0.2f' % metrics[1][1])
    print('F1              %0.2f' % metrics[2][0] + '       %0.2f' % metrics[2][1])

In [14]:
def print_cv(scores):
    fold = [x + 1 for x in range(len(scores['test_precision_macro']))]
    print('         Precision     Recall,   AUC')
    [print_format(f,x,y,z) for f,x,y,z in zip(fold, scores['test_precision_macro'],
                                             scores['test_recall_macro'],
                                             scores['test_roc_auc'])]
    print('-' * 40)
    print('Mean     %4.3f        %4.3f        %4.3f'  %
         (np.mean(scores['test_precision_macro']), np.mean(scores['test_recall_macro']), np.mean(scores['test_roc_auc'])))
    print('Std      %4.3f        %4.3f      %4.3f'    %
         (np.std(scores['test_precision_macro']), np.std(scores['test_recall_macro']), np.std(scores['test_roc_auc'])))
    
#print_cv(scores)

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_curve

rf = RandomForestRegressor(n_estimators=200, max_depth=12, min_samples_split=5)
rf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [27]:
rf.fit(x_train1, y_train1)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [28]:
rf_pred = rf.predict(x_train_val)

In [29]:
from sklearn.metrics import root_me

ValueError: continuous is not supported